**Diana Kim ADA-2403M**

# **Практическая работа №4**
**Тема:** Оптимизация параллельного кода на GPU с использованием различных типов памяти.

**Цель работы:** Изучение и использование различных типов памяти CUDA  (глобальная, разделяемая, локальная) для оптимизации параллельных вычислений на GPU.

In [22]:
!nvidia-smi

Fri Jan  9 09:36:40 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [23]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


***Лабораторная работа разделена на 3 части***

### **1 часть. Редукция суммы, здесь только global memory**

Задача этой части заключается в реализации наивного варианта параллельной редукции суммы элементов массива на GPU с использованием только глобальной памяти. Он используется как базовый эталон для последующего сравнения с оптимизированными решениями, использующими другие типы памяти





In [11]:
%%writefile part1.cu

#include <cuda_runtime.h>        // для функций CUDA, которые мне надо
#include <iostream>                  // ввод/вывод
#include <ctime>        // для time()
#include <cstdlib>        // для rand() и srand()

using namespace std;                 // чтобы не писать std


void cuda_ok(cudaError_t err, const char* msg) {          // функция для проверки ошибок CUDA
    if (err != cudaSuccess) {                  // если есть ошибка
        cout << "CUDA error (" << msg << "): "         // печатает место ошибки
             << cudaGetErrorString(err) << endl;            // печатает текст ошибки
        exit(1);                          // завершает программу
    }
}

__global__ void sum_global(int* data, int* result, int n) {           // ядро суммирует элементы
    int idx = threadIdx.x + blockIdx.x * blockDim.x;            // считает индекс потока
    if (idx < n) {                           // проверяет границы
        atomicAdd(result, data[idx]);                // добавляет элемент в общую сумму
    }
}



int main() {
    int N = 1000000;                     // размер массива
    int size = N * (int)sizeof(int);         // размер массива в байтах
    int* h_data = new int[N];            // выделяет массив на CPU
    int h_result = 0;                         // результат на CPU

    srand((unsigned)time(0));           // запускает генератор случайных чисел

    for (int i = 0; i < N; i++) {          // цикл по массиву
        h_data[i] = rand() % 10;               // заполняет числами от 0 до 9
    }

    int* d_data = nullptr;                   // указатель на массив на GPU
    int* d_result = nullptr;                     // указатель на сумму на GPU
    cuda_ok(cudaMalloc((void**)&d_data, size), "cudaMalloc d_data");            // выделяет память под массив на GPU
    cuda_ok(cudaMalloc((void**)&d_result, (int)sizeof(int)), "cudaMalloc d_result");                 // выделяет память под сумму на GPU
    cuda_ok(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice), "cudaMemcpy H->D data");                  // копирует массив на GPU
    cuda_ok(cudaMemset(d_result, 0, (int)sizeof(int)), "cudaMemset d_result");                    // обнуляет сумму на GPU

    int blockSize = 256;                               // потоки в блоке
    int numBlocks = (N + blockSize - 1) / blockSize;               // блоки в сетке
    sum_global<<<numBlocks, blockSize>>>(d_data, d_result, N);            // запускает ядро
    cuda_ok(cudaGetLastError(), "kernel launch");                // проверяет ошибку запуска ядра
    cuda_ok(cudaDeviceSynchronize(), "cudaDeviceSynchronize");        // ждёт завершения ядра и ловит ошибки выполнения
    cuda_ok(cudaMemcpy(&h_result, d_result, (int)sizeof(int), cudaMemcpyDeviceToHost), "cudaMemcpy D->H result");          // копирует сумму на CPU

    cout << "sum: " << h_result << endl;             // выводит сумму

    cuda_ok(cudaFree(d_data), "cudaFree d_data");            // освобождает память массива на GPU
    cuda_ok(cudaFree(d_result), "cudaFree d_result");           // освобождает память суммы на GPU

    delete[] h_data;                         // освобождает память на CPU

    return 0;
}

Overwriting part1.cu


In [15]:
!nvcc part1.cu -o part1 -arch=compute_75 -code=sm_75

In [16]:
!./part1

sum: 4496561


### **2 часть. Редукция суммы, здесь глобальная память + локальная память**

Задача этой части показать, что использование локальная память позволяет сократить число обращений к медленной глобальной памяти и повысить производительность по сравнению с вариантом, где используется только глобальная память

In [17]:
%%writefile part2.cu

#include <cuda_runtime.h>        // для функций CUDA, которые мне надо
#include <iostream>                  // ввод/вывод
#include <ctime>        // для time()
#include <cstdlib>        // для rand() и srand()

using namespace std;                 // чтобы не писать std


void cuda_ok(cudaError_t err, const char* msg) {            // функция для проверки ошибок CUDA
    if (err != cudaSuccess) {                         // если есть ошибка
        cout << "CUDA error (" << msg << "): "             // печатает место ошибки
             << cudaGetErrorString(err) << endl;              // печатает текст ошибки
        exit(1);                               // завершает программу
    }
}

__global__ void sum_shared(int* data, int* result, int n) {                  // ядро редукция с shared memory
    __shared__ int sharedData[256];                // shared память на блок
    int tid = threadIdx.x;                            // номер потока внутри блока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;         // глобальный индекс элемента
    if (idx < n) {                      // если индекс в пределах массива
        sharedData[tid] = data[idx];           // кладёт элемент из global в shared
    } else {                                             // иначе (если вышли за пределы)
        sharedData[tid] = 0;                 // кладёт 0, чтобы не мешал сумме
    }
    __syncthreads();            // ждёт, пока все потоки запишут sharedData

    for (int s = blockDim.x / 2; s > 0; s /= 2) {           // уменьшает шаг редукции 128,64,32...
        if (tid < s) {                               // только первая половина потоков работает
            sharedData[tid] += sharedData[tid + s];              // складывает пары в shared памяти
        }
        __syncthreads();           // синхронизация после каждого шага
    }
    if (tid == 0) {                         // только поток 0 в блоке
        atomicAdd(result, sharedData[0]);           // добавляет сумму блока в общую сумму global
    }
}



int main() {
    int N = 1000000;                        // размер массива
    int size = N * (int)sizeof(int);             // размер массива в байтах
    int* h_data = new int[N];              // выделяет массив на CPU
    int h_result = 0;                   // результат на CPU
    srand((unsigned)time(0));         // запускает генератор случайных чисел
    for (int i = 0; i < N; i++) {               // цикл по массиву
        h_data[i] = rand() % 10;                  // заполняет числами от 0 до 9
    }

    int* d_data = nullptr;                 // указатель на массив на GPU
    int* d_result = nullptr;           // указатель на сумму на GPU

    cuda_ok(cudaMalloc((void**)&d_data, size), "cudaMalloc d_data");                 // выделяет память массива на GPU
    cuda_ok(cudaMalloc((void**)&d_result, (int)sizeof(int)), "cudaMalloc d_result"); // выделяет память суммы на GPU
    cuda_ok(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice), "cudaMemcpy H->D data"); // копирует массив на GPU
    cuda_ok(cudaMemset(d_result, 0, (int)sizeof(int)), "cudaMemset d_result");                  // обнуляет сумму на GPU

    int blockSize = 256;                      // потоки в блоке ровно под sharedData[256]
    int numBlocks = (N + blockSize - 1) / blockSize;            // количество блоков для покрытия всего массива
    sum_shared<<<numBlocks, blockSize>>>(d_data, d_result, N);            // запускает ядро
    cuda_ok(cudaGetLastError(), "kernel launch");           // проверяет ошибки запуска ядра
    cuda_ok(cudaDeviceSynchronize(), "cudaDeviceSynchronize");                 // ждёт завершения ядра
    cuda_ok(cudaMemcpy(&h_result, d_result, (int)sizeof(int), cudaMemcpyDeviceToHost), "cudaMemcpy D->H result");             // копирует сумму на CPU

    cout << "sum: " << h_result << endl;           // выводит сумму

    cuda_ok(cudaFree(d_data), "cudaFree d_data");                // освобождает память массива на GPU
    cuda_ok(cudaFree(d_result), "cudaFree d_result");                 // освобождает память суммы на GPU

    delete[] h_data;                   // освобождает память на CPU
    return 0;                      // конец программы
}

Writing part2.cu


In [18]:
!nvcc part2.cu -o part2 -arch=compute_75 -code=sm_75

In [19]:
!./part2

sum: 4498584


### **3 часть. Сортировка подмассивов с использованием локальной памяти**

Задача этой части продемонстрировать использование локальной памяти для временных данных внутри одного потока и показать, что локальная память подходит для обработки небольших подмассивов, где важна простота и наглядность алгоритма, а не максимальная производительность.

In [20]:
%%writefile part3.cu

#include <cuda_runtime.h>        // для функций CUDA, которые мне надо
#include <iostream>                  // ввод/вывод
#include <ctime>        // для time()
#include <cstdlib>        // для rand()

using namespace std;            // чтобы не писать std::

void cuda_ok(cudaError_t err, const char* msg) {                     // функция проверки ошибок CUDA
    if (err != cudaSuccess) {                       // если ошибка не равна успеху
        cout << "CUDA error (" << msg << "): " << cudaGetErrorString(err) << endl;             // печатает текст ошибки
        exit(1);                      // завершает программу
    }
}

__global__ void sum_global(int* data, int* result, int n) {               // ядро редукция только через global memory
    int idx = threadIdx.x + blockIdx.x * blockDim.x;            // вычисляет глобальный индекс потока
    if (idx < n) {                   // проверяет границы массива
        atomicAdd(result, data[idx]);                  // добавляет элемент к общей сумме
    }
}

__global__ void sum_shared(int* data, int* result, int n) {                // ядро редукция через shared memory
    __shared__ int sharedData[256];                      // выделяет shared память на блок 256 ints
    int tid = threadIdx.x;                         // номер потока внутри блока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;                   // глобальный индекс элемента
    if (idx < n) {                        // если индекс не выходит за массив
        sharedData[tid] = data[idx];                         // копирует элемент из global в shared
    } else {                    // если индекс вышел за массив
        sharedData[tid] = 0;                    // кладёт 0, чтобы не влиять на сумму
    }
    __syncthreads();                 // ждёт, пока все потоки запишут shared память
    for (int s = blockDim.x / 2; s > 0; s /= 2) {                       // шаги редукции: 128,64,32...
        if (tid < s) {               // работает только первая половина потоков
            sharedData[tid] += sharedData[tid + s];             // складывает значения в shared памяти
        }
        __syncthreads();             // синхронизирует потоки после каждого шага
    }
    if (tid == 0) {                     // только нулевой поток блока
        atomicAdd(result, sharedData[0]);                       // добавляет сумму блока в общую сумму
    }
}


float run_and_time_global(int* h_data, int n, int& out_sum) {                  // функция запуска и замера времени для global версии
    int size = n * (int)sizeof(int);                 // считает размер массива в байтах
    int* d_data = nullptr;                    // указатель на массив на GPU
    int* d_result = nullptr;                             // указатель на сумму на GPU
    cudaEvent_t start, stop;              // переменные для событий CUDA (таймер)
    int h_result = 0;                            // локальная переменная результата на CPU

    cuda_ok(cudaMalloc((void**)&d_data, size), "cudaMalloc d_data (global)");             // выделяет память под массив на GPU
    cuda_ok(cudaMalloc((void**)&d_result, (int)sizeof(int)), "cudaMalloc d_result (global)");              // выделяет память под сумму на GPU
    cuda_ok(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice), "cudaMemcpy H->D data (global)");            // копирует массив на GPU
    cuda_ok(cudaMemset(d_result, 0, (int)sizeof(int)), "cudaMemset d_result (global)");             // обнуляет сумму на GPU

    cuda_ok(cudaEventCreate(&start), "cudaEventCreate start (global)");         // создаёт событие начала
    cuda_ok(cudaEventCreate(&stop), "cudaEventCreate stop (global)");               // создаёт событие конца

    int blockSize = 256;                  // фиксируем размер блока как 256 потоков
    int numBlocks = (n + blockSize - 1) / blockSize;          // считает количество блоков
    cuda_ok(cudaEventRecord(start), "cudaEventRecord start (global)");           // ставит старт таймера
    sum_global<<<numBlocks, blockSize>>>(d_data, d_result, n);             // запускает ядро global версии
    cuda_ok(cudaGetLastError(), "kernel launch (global)");             // проверяет ошибку запуска ядра
    cuda_ok(cudaEventRecord(stop), "cudaEventRecord stop (global)");                  // ставит стоп таймера
    cuda_ok(cudaEventSynchronize(stop), "cudaEventSynchronize stop (global)");               // ждёт завершения и таймера и ядра

    float ms = 0.0f;              // переменная для времени в миллисекундах
    cuda_ok(cudaEventElapsedTime(&ms, start, stop), "cudaEventElapsedTime (global)");               // считает время между началом и концом

    cuda_ok(cudaMemcpy(&h_result, d_result, (int)sizeof(int), cudaMemcpyDeviceToHost), "cudaMemcpy D->H result (global)");       // копирует сумму на CPU
    out_sum = h_result;               // записывает сумму наружу

    cuda_ok(cudaEventDestroy(start), "cudaEventDestroy start (global)");                // удаляет событие начала
    cuda_ok(cudaEventDestroy(stop), "cudaEventDestroy stop (global)");             // удаляет событие конца
    cuda_ok(cudaFree(d_data), "cudaFree d_data (global)");                               // освобождает массив на GPU
    cuda_ok(cudaFree(d_result), "cudaFree d_result (global)");                          // освобождает сумму на GPU

    return ms;                                // возвращает время в миллисекундах
}


float run_and_time_shared(int* h_data, int n, int& out_sum) {                        // функция запуска и замера времени для shared версии
    int size = n * (int)sizeof(int);                // считает размер массива в байтах
    int* d_data = nullptr;                  // указатель на массив на GPU
    int* d_result = nullptr;                         // указатель на сумму на GPU
    cudaEvent_t start, stop;                // переменные для событий CUDA
    int h_result = 0;                           // локальная переменная результата на CPU

    cuda_ok(cudaMalloc((void**)&d_data, size), "cudaMalloc d_data (shared)");                 // выделяет память под массив на GPU
    cuda_ok(cudaMalloc((void**)&d_result, (int)sizeof(int)), "cudaMalloc d_result (shared)");                  // выделяет память под сумму на GPU
    cuda_ok(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice), "cudaMemcpy H->D data (shared)");               // копирует массив на GPU
    cuda_ok(cudaMemset(d_result, 0, (int)sizeof(int)), "cudaMemset d_result (shared)");                 // обнуляет сумму на GPU

    cuda_ok(cudaEventCreate(&start), "cudaEventCreate start (shared)");                  // создаёт событие начала
    cuda_ok(cudaEventCreate(&stop), "cudaEventCreate stop (shared)");           // создаёт событие конца

    int blockSize = 256;                    // фиксируем размер блока как 256 потоков
    int numBlocks = (n + blockSize - 1) / blockSize;                 // считает количество блоков

    cuda_ok(cudaEventRecord(start), "cudaEventRecord start (shared)");               // ставит старт таймера
    sum_shared<<<numBlocks, blockSize>>>(d_data, d_result, n);            // запускает ядро shared версии
    cuda_ok(cudaGetLastError(), "kernel launch (shared)");                     // проверяет ошибку запуска ядра
    cuda_ok(cudaEventRecord(stop), "cudaEventRecord stop (shared)");                   // ставит стоп таймера
    cuda_ok(cudaEventSynchronize(stop), "cudaEventSynchronize stop (shared)");                    // ждёт завершения и таймера и ядра

    float ms = 0.0f;                      // переменная для времени в миллисекундах
    cuda_ok(cudaEventElapsedTime(&ms, start, stop), "cudaEventElapsedTime (shared)");                  // считает время между start и stop

    cuda_ok(cudaMemcpy(&h_result, d_result, (int)sizeof(int), cudaMemcpyDeviceToHost), "cudaMemcpy D->H result (shared)");       // копирует сумму на CPU
    out_sum = h_result;             // записывает сумму наружу

    cuda_ok(cudaEventDestroy(start), "cudaEventDestroy start (shared)");                 // удаляет событие начала
    cuda_ok(cudaEventDestroy(stop), "cudaEventDestroy stop (shared)");                // удаляет событие конца
    cuda_ok(cudaFree(d_data), "cudaFree d_data (shared)");                   // освобождает массив на GPU
    cuda_ok(cudaFree(d_result), "cudaFree d_result (shared)");                          // освобождает сумму на GPU

    return ms;                           // возвращает время в миллисекундах
}



int main() {
    srand((unsigned)time(0));              // инициализирует генератор случайных чисел
    int sizes[3] = {10000, 100000, 1000000};                 // массив размеров для теста
    cout << "N | global ms | shared ms | sum_global | sum_shared" << endl; // печатает заголовок таблицы

    for (int t = 0; t < 3; t++) {                     // цикл по трем размерам
        int N = sizes[t];                       // берёт текущий размер
        int* h_data = new int[N];                      // выделяет массив на CPU

        for (int i = 0; i < N; i++) {                  // заполняет массив
            h_data[i] = rand() % 10;             // кладёт число от 0 до 9
        }

        int sum1 = 0;                 // сумма для global версии
        int sum2 = 0;                     // сумма для shared версии

        float ms_global = run_and_time_global(h_data, N, sum1);                // запускает global и получает время и сумму
        float ms_shared = run_and_time_shared(h_data, N, sum2);                   // запускает shared и получает время и сумму

        cout << N << " | " << ms_global << " | " << ms_shared << " | " << sum1 << " | " << sum2 << endl;           // печатает строку результатов

        delete[] h_data;             // освобождает массив на CPU
    }

    return 0;
}

Writing part3.cu


In [21]:
!nvcc part3.cu -o part3 -arch=compute_75 -code=sm_75

In [22]:
!./part3

N | global ms | shared ms | sum_global | sum_shared
10000 | 0.1312 | 0.026304 | 45043 | 45043
100000 | 0.010752 | 0.016096 | 450700 | 450700
1000000 | 0.05328 | 0.100992 | 4499354 | 4499354
